In [149]:
import pandas as pd
import numpy as np

In [150]:
# Open database
df_1 = pd.read_excel("rbf_data_1.xlsx")
df_2 = pd.read_excel("rbf_data_2.xlsx")
df_3 = pd.read_excel("rbf_data_3.xlsx")
df_4 = pd.read_excel("rbf_data_4.xlsx")

# 1
df_op_1 = df_1.drop(columns=["REFERENCE"])
column_ref_1 = df_1["REFERENCE"].astype(float)
print(df_op_1)
print(column_ref_1)

# 2
df_op_2 = df_2.drop(columns=["REFERENCE"])
column_ref_2 = df_2["REFERENCE"].astype(float)
print(df_op_2)
print(column_ref_2)

# 3
df_op_3 = df_3.drop(columns=["REFERENCE"])
column_ref_3 = df_3["REFERENCE"].astype(float)
print(df_op_3)
print(column_ref_3)

# 4
df_4.iloc[6, 3] = 2.11
df_op_4 = df_4.drop(columns=["REFERENCE"])
column_ref_4 = df_4["REFERENCE"].astype(float)
print(df_op_4)
print(column_ref_4)

   X1  X2  X3  X4  X5  X6  X7  X8  X9  X10
0   1   0   1   0   1   1   0   1   0    1
1   1   0   1   0   0   1   1   1   0    1
2   0   1   1   1   0   1   0   1   1    0
3   0   1   1   0   1   1   0   0   1    1
4   1   0   0   0   1   1   1   1   1    1
5   1   1   1   1   0   0   0   0   0    1
6   1   1   0   1   0   1   1   0   1    0
7   0   1   0   1   0   1   0   1   0    1
8   1   1   1   0   0   1   1   0   0    1
0    10.0
1   -10.0
2   -10.0
3    10.0
4   -10.0
5   -10.0
6    10.0
7    10.0
8   -10.0
Name: REFERENCE, dtype: float64
   X1
0   2
1   3
2   4
0    3.0
1    6.0
2    5.0
Name: REFERENCE, dtype: float64
   X1  X2
0   1   1
1   0   1
2   0   0
3   1   0
0    0.0
1    1.0
2    0.0
3    1.0
Name: REFERENCE, dtype: float64
   X1  X2  X3
0   1   1   1
1   1   1   0
2   0   3   1
3   1   1   2
4   2   1   1
5   2   0   4
6   1   2   0
0    2.18
1    1.83
2    1.31
3    2.79
4    2.49
5    3.95
6    2.11
Name: REFERENCE, dtype: float64


In [151]:
def oil_database(df_1, df_2, df_3, df_4):

	# 1
	df_op_1 = df_1.drop(columns=["REFERENCE"])
	column_ref_1 = df_1["REFERENCE"].astype(float)
	
	# 2
	df_op_2 = df_2.drop(columns=["REFERENCE"])
	column_ref_2 = df_2["REFERENCE"].astype(float)
	
	# 3
	df_op_3 = df_3.drop(columns=["REFERENCE"])
	column_ref_3 = df_3["REFERENCE"].astype(float)
	
	# 4
	df_4.iloc[6, 3] = 2.11
	df_op_4 = df_4.drop(columns=["REFERENCE"])
	column_ref_4 = df_4["REFERENCE"].astype(float)
	
	return df_op_1, df_op_2, df_op_3, df_op_4, \
		   column_ref_1, column_ref_2, column_ref_3, column_ref_4

In [152]:
def select_two_poles(df_op):
	pole_size = len(df_op.columns)
	pole_zero = np.zeros(pole_size)
	pole_one = df_op.max().values
	nbr_of_poles = 2
	return nbr_of_poles, pole_zero, pole_one

In [153]:
def calculate_euclidian_distance(pole_zero, pole_one):
	pole_distance = np.linalg.norm(pole_one - pole_zero)**2
	return pole_distance

In [154]:
def calculate_sigma(nbr_of_poles, pole_distance):
	sigma = np.sqrt(pole_distance) / np.sqrt(2 * nbr_of_poles)
	return sigma

In [155]:
def gaussian_function(row, pole_zero, pole_one, sigma):
	result1 = np.e**(-1/(2*(sigma**2)) * np.linalg.norm(row - pole_zero)**2)
	result2 = np.e**(-1/(2*(sigma**2)) * np.linalg.norm(row - pole_one)**2)
	return result1, result2

def apply_radial_basis_function_in_database(df_op, pole_zero, pole_one, sigma):
	tuple_series = df_op.apply(gaussian_function, axis=1, args=(pole_zero, pole_one, sigma))
	df_main_matrix = pd.DataFrame(tuple_series.tolist(), columns=['Column 1', 'Column 2'])
	df_main_matrix["Column 3"] = 1
	R = df_main_matrix.values
	return R

In [156]:
def calculate_pseudo_inverse(R):
	R_pseudo_inv = np.dot(np.linalg.inv(np.dot(R.T, R)), R.T)
	return R_pseudo_inv

In [157]:
def transform_column_ref_in_matrix(column_ref):
	A = column_ref.to_numpy().reshape(-1, 1)
	return A

In [158]:
def calculate_radial_basis_function_constants(R_pseudo_inv, A):
	a = np.dot(R_pseudo_inv, A)
	return a

In [159]:
def truth_gaussian_function(row, pole_zero, pole_one, sigma, a):
	first_term = a[0] * np.e**(-1/(2*(sigma**2)) * np.linalg.norm(row - pole_zero)**2)
	second_term = a[1] * np.e**(-1/(2*(sigma**2)) * np.linalg.norm(row - pole_one)**2)
	third_term = a[2]
	result = first_term + second_term + third_term
	return result

def predict_new_values(new_df_op, pole_zero, pole_one, sigma, a):
	result = new_df_op.apply(truth_gaussian_function, axis=1, args=(pole_zero, pole_one, sigma, a))
	return result

In [164]:
# main function
# Open database
df_1 = pd.read_excel("rbf_data_1.xlsx")
df_2 = pd.read_excel("rbf_data_2.xlsx")
df_3 = pd.read_excel("rbf_data_3.xlsx")
df_4 = pd.read_excel("rbf_data_4.xlsx")

# Treat database
df_op_1, df_op_2, df_op_3, df_op_4, \
column_ref_1, column_ref_2, column_ref_3, column_ref_4 = \
oil_database(df_1, df_2, df_3, df_4)

# Chose datas to be processed
df_op = df_op_2
column_ref = column_ref_2

# Algorithm core
nbr_of_poles, pole_zero, pole_one = select_two_poles(df_op)
pole_distance = calculate_euclidian_distance(pole_zero, pole_one)
sigma = calculate_sigma(nbr_of_poles, pole_distance)
R = apply_radial_basis_function_in_database(df_op, pole_zero, pole_one, sigma)
R_pseudo_inv = calculate_pseudo_inverse(R)
A = transform_column_ref_in_matrix(column_ref)
a = calculate_radial_basis_function_constants(R_pseudo_inv, A)

# Define datas to be predicted
df_op_to_predic = df_op

# Predict data reference
prediction = predict_new_values(df_op_to_predic, pole_zero, pole_one, sigma, a)
print(prediction)

0    [2.9999999999999076]
1     [6.000000000000526]
2     [5.000000000000689]
dtype: object
